In [1]:
%reload_ext autoreload
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.functional as F
import torchvision
import matplotlib.pyplot as plt
from pprint import pprint
import numpy as np
from numpy.linalg import inv
from numpy.linalg import multi_dot as mdot
from IPython.core.debugger import set_trace

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cup")
device

device(type='cuda', index=0)

In [6]:
import random
import datetime
from ppt import utils
from ppt.utils import attr

In [10]:
from torchvision import transforms

IMG_SIZE = 224
_mean = [0.485, 0.456, 0.406]
_std = [0.229, 0.224, 0.225]

train_trans = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(.3,.3,.3),
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])
val_trans = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])

In [11]:
from ppt.utils import DogsCatsDataset

In [13]:
train_ds = DogsCatsDataset(r"C:\Users\tensorflow\Desktop\iloraz_inteligencji\torch\data\raw", r"sample/train", transform=train_trans)
val_ds = DogsCatsDataset(r"C:\Users\tensorflow\Desktop\iloraz_inteligencji\torch\data\raw", r"sample/valid", transform=val_trans)

Loading data from C:\Users\tensorflow\Desktop\iloraz_inteligencji\torch\data\raw\dogscats\sample/train.
Loading data from C:\Users\tensorflow\Desktop\iloraz_inteligencji\torch\data\raw\dogscats\sample/valid.


In [14]:
BATCH_SIZE=2
n_classes = 2

In [ ]:
# train_ds = DogsCatsDataset(r"C:\Users\tensorflow\Desktop\iloraz_inteligencji\torch\data\raw", r"train", transform=train_trans)
# val_ds = DogsCatsDataset(r"C:\Users\tensorflow\Desktop\iloraz_inteligencji\torch\data\raw", r"valid", transform=val_trans)
# BATCH_SIZE=512

In [54]:
from torch.utils.data import DataLoader

train_dl = DataLoader(
    train_ds,
    batch_size = BATCH_SIZE,
    shuffle=True,
    num_workers=1,
)
val_dl = DataLoader(
    val_ds,
    batch_size = BATCH_SIZE,
    shuffle=False,
    num_workers=1,
)

In [55]:
import torchvision

In [56]:
from torchvision import models
model = models.resnet18(pretrained=True)

In [57]:
#model

In [58]:
# frize all params
for param in model.parameters():
    param.requires_grad =False

In [59]:
# trainig helpers
def get_trainable(model_params):
    return(p for p in model_params if p.requires_grad)

def get_frozen(model_params):
    return(p for p in model_params if not p.requires_grad)

def all_trainable(model_params):
    return all(p.requires_grad for p in model_params)
# check if all params are frozen
def all_frozen(model_params):
    return all(not p.requires_grad for p in model_params)
# freez all layers 
def freeze_all(model_params):
    for param in model_params:
        param.requires_grad = False
        
#list(get_trainable(model.parameters()))
#list(get_frozen(model.parameters()))
# all_trainable(model.parameters())
# all_frozen(model.parameters())

In [60]:
freeze_all(model.parameters())
assert all_frozen(model.parameters())

Replays last layer

In [61]:
model.fc = nn.Linear(512, n_classes)

In [62]:
all_frozen(model.parameters())

False

In [63]:
def get_model(n_classes =2):
    model = models.resnet18(pretrained=True)
    freeze_all(model.parameters())
    model.fc = nn.Linear(512, n_classes)
    return model

model = get_model().to(device)

Define loss

In [64]:
criterion = nn.CrossEntropyLoss()

In [65]:
optimizer = torch.optim.Adam(
    get_trainable(model.parameters()),
    lr = 0.001,
    #momentum =0.9,
    #model.fc.parameters(),
)

In [66]:
N_EPOCHS = 2

In [68]:
%%time
for epoch in range(N_EPOCHS):
    print(f"Epoch {epoch+1}/{N_EPOCHS}")
    model.train()
    running_loss, correct = 0.0, 0
    for X, y in train_dl:# we iterate over training data
        X, y = X.to(device), y.to(device)# data do to gpu
        optimizer.zero_grad()# zeroje gradienty
        y_ = model(X)# data to model
        loss = criterion(y_, y)# calculate loss
        loss.backward()
        optimizer.step()# call optimazer
        # stats
        print(f"    batch loss: {loss.item():0.3f}")
        _, y_label_ = torch.max(y_,1)
        correct += (y_label_ == y).sum().item()
        running_loss += loss.item()*X.shape[0]
        
    print(f"  Train Loss: {running_loss / len(train_dl.dataset)}")
    print(f"  Train Acc:  {correct / len(train_dl.dataset)}")
    
    # so now validation set
    model.eval()
    running_loss, correct = 0.0, 0
    with torch.no_grad():
        for X, y in val_dl:
            X, y = X.to(device), y.to(device)
            y_ = model(X)
            _, y_label_ = torch.max(y_,1)
            correct += (y_label_ == y).sum().item()
            loss = criterion(y_, y)
            running_loss += loss.item()*X.shape[0]
            
    print(f"  Valid Loss: {running_loss / len(val_dl.dataset)}")
    print(f"  Valid Acc:  {correct / len(val_dl.dataset)}")
    print()

Epoch 1/2
    batch loss: 0.735
    batch loss: 0.631
    batch loss: 0.528
    batch loss: 0.456
    batch loss: 0.436
    batch loss: 0.901
    batch loss: 0.593
    batch loss: 0.398
  Train Loss: 0.5847767405211926
  Train Acc:  0.875
  Valid Loss: 0.4822899326682091
  Valid Acc:  0.875

Epoch 2/2
    batch loss: 0.252
    batch loss: 0.275
    batch loss: 0.623
    batch loss: 0.926
    batch loss: 0.415
    batch loss: 0.646
    batch loss: 0.771
    batch loss: 0.733
  Train Loss: 0.5799270533025265
  Train Acc:  0.5625
  Valid Loss: 0.4832411855459213
  Valid Acc:  0.875

Wall time: 4.23 s


# traing liberys

# ignite

In [84]:
import ignite
from ignite import CategoricalAccuracy

ImportError: cannot import name 'CategoricalAccuracy'